# HLW-COVID Brasil - Exploração de Dados

Este notebook realiza a exploração inicial dos dados econômicos para o modelo Holston-Laubach-Williams adaptado ao Brasil com ajustes COVID.

## Dados analisados:
- PIB Real Trimestral
- IPCA (Inflação)
- Taxa Selic
- Oxford Stringency Index

## 1. Setup e Importações

In [ ]:
# Standard library
import sys
from pathlib import Path
import logging

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Scientific computing
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Statistical tests
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Project modules
from src.data.loader import DataLoader
from src.data.preprocessing import DataPreprocessor
from src.utils.validation import (
    validate_quarterly_data,
    check_missing_values,
    validate_model_data,
)

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)

# Plot settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Imports successful")

## 2. Carregamento de Dados

In [ ]:
# Initialize data loader
config_path = project_root / "config" / "config.yaml"
loader = DataLoader(config_path)

print(f"Configuration loaded from: {config_path}")

In [ ]:
# Load all data sources
# Note: This will fail if data files don't exist yet
# You'll need to add your data files to the data/raw/ directory

try:
    raw_data = loader.load_all()
    print("\n✓ Data loaded successfully")
    print(f"\nAvailable datasets: {list(raw_data.keys())}")
except Exception as e:
    print(f"❌ Error loading data: {e}")
    print("\nMake sure to add your data files to:")
    print("  - data/raw/pib_trimestral.xlsm")
    print("  - data/raw/ipca.xlsm")
    print("  - data/raw/selic.xlsm")
    print("  - data/external/oxford_stringency_index.csv")

## 3. Estatísticas Descritivas

In [ ]:
# PIB Real
if raw_data.get('pib') is not None:
    print("=" * 60)
    print("PIB REAL TRIMESTRAL")
    print("=" * 60)
    print(f"\nPeríodo: {raw_data['pib']['date'].min()} a {raw_data['pib']['date'].max()}")
    print(f"Observações: {len(raw_data['pib'])}")
    print("\nEstatísticas:")
    print(raw_data['pib']['pib_real'].describe())
    print(f"\nValores faltantes: {raw_data['pib']['pib_real'].isnull().sum()}")

In [ ]:
# IPCA
if raw_data.get('ipca') is not None:
    print("=" * 60)
    print("IPCA (INFLAÇÃO)")
    print("=" * 60)
    print(f"\nPeríodo: {raw_data['ipca']['date'].min()} a {raw_data['ipca']['date'].max()}")
    print(f"Observações: {len(raw_data['ipca'])}")
    print("\nEstatísticas - IPCA Núcleo:")
    print(raw_data['ipca']['ipca_nucleo'].describe())
    print("\nEstatísticas - IPCA Cheio:")
    print(raw_data['ipca']['ipca_cheio'].describe())

In [ ]:
# Selic
if raw_data.get('selic') is not None:
    print("=" * 60)
    print("TAXA SELIC")
    print("=" * 60)
    print(f"\nPeríodo: {raw_data['selic']['date'].min()} a {raw_data['selic']['date'].max()}")
    print(f"Observações: {len(raw_data['selic'])}")
    print("\nEstatísticas:")
    print(raw_data['selic']['selic_nominal'].describe())

In [ ]:
# Stringency Index
if raw_data.get('stringency') is not None:
    print("=" * 60)
    print("OXFORD STRINGENCY INDEX")
    print("=" * 60)
    print(f"\nPeríodo: {raw_data['stringency']['date'].min()} a {raw_data['stringency']['date'].max()}")
    print(f"Observações: {len(raw_data['stringency'])}")
    print("\nEstatísticas:")
    print(raw_data['stringency']['stringency_index'].describe())

## 4. Visualizações Exploratórias

In [ ]:
# PIB Real - Série temporal
if raw_data.get('pib') is not None:
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(raw_data['pib']['date'], raw_data['pib']['pib_real'], linewidth=2)
    ax.set_title('PIB Real Trimestral - Brasil', fontsize=14, fontweight='bold')
    ax.set_xlabel('Data')
    ax.set_ylabel('PIB Real (R$ milhões)')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# IPCA - Núcleo vs Cheio
if raw_data.get('ipca') is not None:
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(raw_data['ipca']['date'], raw_data['ipca']['ipca_nucleo'], 
            label='IPCA Núcleo', linewidth=2)
    ax.plot(raw_data['ipca']['date'], raw_data['ipca']['ipca_cheio'], 
            label='IPCA Cheio', linewidth=2, alpha=0.7)
    ax.set_title('IPCA - Núcleo vs Cheio', fontsize=14, fontweight='bold')
    ax.set_xlabel('Data')
    ax.set_ylabel('Variação % (a.a.)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.axhline(y=0, color='black', linestyle='--', linewidth=0.8)
    plt.tight_layout()
    plt.show()

In [ ]:
# Selic
if raw_data.get('selic') is not None:
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(raw_data['selic']['date'], raw_data['selic']['selic_nominal'], 
            linewidth=2, color='darkred')
    ax.set_title('Taxa Selic Nominal', fontsize=14, fontweight='bold')
    ax.set_xlabel('Data')
    ax.set_ylabel('Taxa (%)')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

In [ ]:
# Stringency Index
if raw_data.get('stringency') is not None:
    fig, ax = plt.subplots(figsize=(12, 6))
    ax.plot(raw_data['stringency']['date'], 
            raw_data['stringency']['stringency_index'], 
            linewidth=2, color='darkgreen')
    ax.set_title('Oxford Stringency Index - Brasil', fontsize=14, fontweight='bold')
    ax.set_xlabel('Data')
    ax.set_ylabel('Índice (0-100)')
    ax.axvline(x=pd.Timestamp('2020-03-01'), color='red', linestyle='--', 
               linewidth=1.5, label='Início COVID')
    ax.legend()
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

## 5. Pré-processamento e Criação das Variáveis do Modelo

In [ ]:
# Initialize preprocessor
preprocessor = DataPreprocessor(config_path)

# Create model data
try:
    model_data = preprocessor.create_model_data(raw_data)
    print("\n✓ Model data created successfully")
    print(f"\nShape: {model_data.shape}")
    print(f"\nColumns: {model_data.columns.tolist()}")
except Exception as e:
    print(f"❌ Error creating model data: {e}")

In [ ]:
# Display first and last rows
print("Primeiras observações:")
display(model_data.head(10))

print("\nÚltimas observações:")
display(model_data.tail(10))

In [ ]:
# Summary statistics of model variables
print("=" * 60)
print("ESTATÍSTICAS DAS VARIÁVEIS DO MODELO")
print("=" * 60)
display(model_data.describe())

## 6. Validação dos Dados

In [ ]:
# Validate quarterly frequency
is_valid, msg = validate_quarterly_data(model_data)
print(f"Frequência trimestral: {'✓' if is_valid else '❌'} {msg}")

In [ ]:
# Check missing values
is_valid, missing_info = check_missing_values(model_data, max_missing_pct=0.05)
print(f"\nValores faltantes: {'✓ OK' if is_valid else '❌ Problemas encontrados'}")
print("\nPercentual de valores faltantes por variável:")
for col, pct in missing_info.items():
    status = '✓' if pct <= 0.05 else '❌'
    print(f"  {status} {col}: {pct:.2%}")

In [ ]:
# Comprehensive validation
is_valid, issues = validate_model_data(model_data, config=loader.config)
if is_valid:
    print("\n✓ Todos os testes de validação passaram!")
else:
    print("\n❌ Problemas encontrados:")
    for issue in issues:
        print(f"  - {issue}")

## 7. Testes de Estacionariedade

In [ ]:
def test_stationarity(series, name):
    """
    Perform ADF and KPSS tests for stationarity.
    """
    # Remove NaN values
    series_clean = series.dropna()
    
    print(f"\n{'=' * 60}")
    print(f"TESTES DE ESTACIONARIEDADE: {name}")
    print(f"{'=' * 60}")
    
    # Augmented Dickey-Fuller test
    print("\nAugmented Dickey-Fuller Test:")
    adf_result = adfuller(series_clean, autolag='AIC')
    print(f"  ADF Statistic: {adf_result[0]:.4f}")
    print(f"  p-value: {adf_result[1]:.4f}")
    print(f"  Critical values:")
    for key, value in adf_result[4].items():
        print(f"    {key}: {value:.4f}")
    
    if adf_result[1] <= 0.05:
        print("  ✓ Rejeita H0: Série é ESTACIONÁRIA (ADF)")
    else:
        print("  ❌ Não rejeita H0: Série é NÃO-ESTACIONÁRIA (ADF)")
    
    # KPSS test
    print("\nKPSS Test:")
    kpss_result = kpss(series_clean, regression='c', nlags='auto')
    print(f"  KPSS Statistic: {kpss_result[0]:.4f}")
    print(f"  p-value: {kpss_result[1]:.4f}")
    print(f"  Critical values:")
    for key, value in kpss_result[3].items():
        print(f"    {key}: {value:.4f}")
    
    if kpss_result[1] >= 0.05:
        print("  ✓ Não rejeita H0: Série é ESTACIONÁRIA (KPSS)")
    else:
        print("  ❌ Rejeita H0: Série é NÃO-ESTACIONÁRIA (KPSS)")

In [ ]:
# Test key variables
if 'y_t' in model_data.columns:
    test_stationarity(model_data['y_t'], 'Log PIB (y_t)')

In [ ]:
if 'pi_t' in model_data.columns:
    test_stationarity(model_data['pi_t'], 'Inflação (π_t)')

In [ ]:
if 'r_t' in model_data.columns:
    test_stationarity(model_data['r_t'], 'Taxa Real (r_t)')

## 8. Análise do Período COVID

In [ ]:
# Plot all variables with COVID period highlighted
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

covid_start = pd.Timestamp('2020-04-01')
covid_end = pd.Timestamp('2022-12-31')

# y_t
if 'y_t' in model_data.columns:
    axes[0].plot(model_data['date'], model_data['y_t'], linewidth=2)
    axes[0].axvspan(covid_start, covid_end, alpha=0.2, color='red')
    axes[0].set_title('Log PIB Real (y_t)', fontweight='bold')
    axes[0].set_ylabel('Log(PIB)')
    axes[0].grid(True, alpha=0.3)

# π_t
if 'pi_t' in model_data.columns:
    axes[1].plot(model_data['date'], model_data['pi_t'], linewidth=2, color='orange')
    axes[1].axvspan(covid_start, covid_end, alpha=0.2, color='red')
    axes[1].set_title('Inflação Trimestral Anualizada (π_t)', fontweight='bold')
    axes[1].set_ylabel('π_t (%)')
    axes[1].grid(True, alpha=0.3)

# r_t
if 'r_t' in model_data.columns:
    axes[2].plot(model_data['date'], model_data['r_t'], linewidth=2, color='darkred')
    axes[2].axvspan(covid_start, covid_end, alpha=0.2, color='red')
    axes[2].set_title('Taxa de Juros Real Ex-Ante (r_t)', fontweight='bold')
    axes[2].set_ylabel('r_t (%)')
    axes[2].grid(True, alpha=0.3)

# d_t
if 'd_t' in model_data.columns:
    axes[3].plot(model_data['date'], model_data['d_t'], linewidth=2, color='darkgreen')
    axes[3].axvspan(covid_start, covid_end, alpha=0.2, color='red')
    axes[3].set_title('Stringency Index (d_t)', fontweight='bold')
    axes[3].set_ylabel('Índice (0-100)')
    axes[3].set_xlabel('Data')
    axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 9. Exportar Dados Processados

In [ ]:
# Save processed data
output_path = project_root / "data" / "processed" / "model_data.csv"
model_data.to_csv(output_path, index=False)
print(f"\n✓ Dados processados salvos em: {output_path}")

## 10. Resumo e Próximos Passos

In [ ]:
print("=" * 60)
print("RESUMO DA EXPLORAÇÃO DE DADOS")
print("=" * 60)
print(f"\nPeríodo da amostra: {model_data['date'].min().date()} a {model_data['date'].max().date()}")
print(f"Número de observações: {len(model_data)}")
print(f"\nVariáveis criadas: {[col for col in model_data.columns if col != 'date']}")
print(f"\nDummies COVID: D_2020, D_2021, D_2022")
print(f"\nPróximos passos:")
print("  1. Especificar o modelo de espaço de estados (Filtro de Kalman)")
print("  2. Implementar função de log-verossimilhança")
print("  3. Estimar parâmetros via MLE")
print("  4. Extrair r* (taxa natural de juros)")
print("  5. Analisar resultados e comparar com literatura")